In [20]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.api import VAR
from datetime import datetime, timedelta
import logging

logging.basicConfig(level=logging.INFO)

class StraddleSelector:
    def __init__(self, equities, lag_period):
        self.equities = equities
        self.lag_period = lag_period
        self.var_models = {}
        self.results = []
        self.p_values_df = pd.DataFrame(columns=['Ticker1', 'Ticker2', 'P_Value', 'Correlation_Significant', 'Var_Model'])
        self.initial_calculation_done = False

    def fetch_historical_data(self, ticker):
        return yf.Ticker(ticker).history(period="1y")

    def calculate_returns(self, historical_data):
        return np.log(historical_data['Close'] / historical_data['Close'].shift(1)).dropna()

    def check_cointegration(self, hist1, hist2):
        _, p_value, _ = coint(hist1['Close'], hist2['Close'])
        return p_value < 0.05, p_value

    def calculate_rolling_correlation(self, returns1, returns2):
        return rolling_correlation(returns1, returns2, window=self.lag_period)

    def fit_var_model(self, returns1, returns2, ticker1, ticker2):
        model_data = pd.concat([returns1, returns2], axis=1).dropna()
        model_data.columns = [ticker1, ticker2]
        var_model = VAR(model_data)
        return var_model.fit(maxlags=15, ic='aic')

    def store_p_values_and_models(self, ticker1, ticker2, p_value, correlation_significant, var_result):
        new_row = pd.DataFrame({
            'Ticker1': [ticker1],
            'Ticker2': [ticker2],
            'P_Value': [p_value],
            'Correlation_Significant': [correlation_significant],
            'Var_Model': [var_result]
        })
        self.p_values_df = pd.concat([self.p_values_df, new_row], ignore_index=True)
        self.var_models[(ticker1, ticker2)] = var_result

    def initial_calculations(self):
        for i, ticker1 in enumerate(self.equities):
            for ticker2 in self.equities[i+1:]:
                logging.info(f"Initial analysis for pair: {ticker1}, {ticker2}")

                hist1 = self.fetch_historical_data(ticker1)
                hist2 = self.fetch_historical_data(ticker2)

                if hist1.empty or hist2.empty:
                    logging.warning(f"No data for pair: {ticker1}, {ticker2}")
                    continue

                returns1 = self.calculate_returns(hist1)
                returns2 = self.calculate_returns(hist2)

                correlation_significant, p_value = self.check_cointegration(hist1, hist2)
                rolling_corr = self.calculate_rolling_correlation(returns1, returns2)
                significant = test_significance(rolling_corr.dropna())

                if not significant:
                    continue

                var_result = self.fit_var_model(returns1, returns2, ticker1, ticker2)
                self.store_p_values_and_models(ticker1, ticker2, p_value, correlation_significant, var_result)

        self.initial_calculation_done = True

    def fetch_latest_price(self, ticker):
        latest_price = yf.Ticker(ticker).history(period="1d")
        if latest_price.empty:
            logging.warning(f"No latest price data for {ticker}")
            return None
        return latest_price['Close'].iloc[-1]

    def forecast_volatility_change(self, var_result):
        try:
            model_data = var_result.endog
            lag_order = var_result.k_ar

            if len(model_data) >= lag_order:
                forecast_input = model_data[-lag_order:]
                forecast = var_result.forecast(y=forecast_input, steps=1)
                vol_change = (forecast[0, 1] - forecast[0, 0]).item() # Difference in predicted returns
                std_change = float(np.std(forecast))
                return vol_change, std_change
            else:
                logging.warning(f"Not enough data points to perform forecasting")
                return 0.0, 0.0
        except Exception as e:
            logging.error(f"Error forecasting volatility change: {e}")
            return 0.0, 0.0

    def update_decisions(self):
        if not self.initial_calculation_done:
            logging.info("Initial calculations not done yet.")
            return

        for (ticker1, ticker2), var_result in self.var_models.items():
            logging.info(f"Updating decisions for pair: {ticker1}, {ticker2}")

            S = self.fetch_latest_price(ticker1)
            if S is None:
                continue

            vol_change, std_change = self.forecast_volatility_change(var_result)
            predicted_mean_change, predicted_std_change = bayesian_inference(S, vol_change, std_change)

            expirations = yf.Ticker(ticker1).options
            self.evaluate_straddle_options(ticker1, ticker2, S, expirations, predicted_mean_change)

    def evaluate_straddle_options(self, ticker1, ticker2, S, expirations, predicted_mean_change):
        pair_results = []

        for expiration in expirations:
            T = (pd.to_datetime(expiration) - pd.Timestamp.today()).days / 365.0

            hist1 = self.fetch_historical_data(ticker1)
            returns1 = self.calculate_returns(hist1)
            sigma = returns1.std() * np.sqrt(252)

            strike_min = S * 0.8
            strike_max = S * 1.2
            initial_guess = S

            result = minimize(objective, initial_guess, args=(S, T, 0.05, sigma), bounds=[(strike_min, strike_max)], method='L-BFGS-B')
            K_opt = result.x[0]

            profit_up, profit_down, profit_no_move, total_cost = straddle_profit(S, K_opt, T, 0.05, sigma)
            profits = [profit_up, profit_down, profit_no_move]

            pair_results.append({
                'Expiration': expiration,
                'Strike': K_opt,
                'Total_Cost': total_cost,
                'Profit_Up': profit_up,
                'Profit_Down': profit_down,
                'Profit_No_Move': profit_no_move,
                'Profits': profits,
                'Sharpe_Ratio': sharpe_ratio(profits, 0.05),
                'Ticker1': ticker1,
                'Ticker2': ticker2
            })

        self.results.extend(pair_results)
        self.select_best_straddle(pair_results, predicted_mean_change)

    def select_best_straddle(self, pair_results, predicted_mean_change):
        best_result = max(pair_results, key=lambda x: x['Sharpe_Ratio'])

        threshold = 0.05
        if predicted_mean_change > threshold:
            logging.info(f"Best Straddle - Expiration: {best_result['Expiration']}, Strike: {best_result['Strike']}, Total Cost: {best_result['Total_Cost']}")
            logging.info(f"Profit if price moves up: {best_result['Profit_Up']}, Profit if price moves down: {best_result['Profit_Down']}, Profit if no movement: {best_result['Profit_No_Move']}")
            logging.info(f"Sharpe Ratio: {best_result['Sharpe_Ratio']}")
        else:
            logging.info("Predicted price change does not meet the threshold for making a purchase.")

    def run_iterations(self, num_iterations):
        for i in range(num_iterations):
            logging.info(f"Running iteration {i + 1}/{num_iterations}")
            self.update_decisions()
            logging.info(f"Iteration {i + 1} completed")
            # Print the results for the current iteration
            for result in self.results:
                logging.info(result)

def rolling_correlation(data1, data2, window=30):
    return data1.rolling(window).corr(data2)

def test_significance(correlation_series, alpha=0.05):
    mean_corr = correlation_series.mean()
    std_corr = correlation_series.std()
    t_stat = mean_corr / (std_corr / np.sqrt(len(correlation_series)))
    p_value = 2 * (1 - norm.cdf(np.abs(t_stat)))  # two-tailed test
    return p_value < alpha

def bayesian_inference(S, vol_change, std_change, n=10000):
    predicted_changes = np.random.normal(loc=vol_change, scale=std_change, size=n)
    future_prices = S * (1 + predicted_changes)
    return future_prices.mean(), future_prices.std()


def binomial_tree_american(S, K, T, r, sigma, option_type='call', steps=100):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    option_values = np.zeros((steps + 1, steps + 1))
    
    # Initialize the option values at maturity
    for i in range(steps + 1):
        if option_type == 'call':
            option_values[i, steps] = max(0, S * (u ** (steps - i)) * (d ** i) - K)
        elif option_type == 'put':
            option_values[i, steps] = max(0, K - S * (u ** (steps - i)) * (d ** i))
    
    # Step back through the tree
    for j in range(steps - 1, -1, -1):
        for i in range(j + 1):
            option_value_if_held = np.exp(-r * dt) * (p * option_values[i, j + 1] + (1 - p) * option_values[i + 1, j + 1])
            if option_type == 'call':
                option_value_if_exercised = S * (u ** i) * (d ** (j - i)) - K
            elif option_type == 'put':
                option_value_if_exercised = K - S * (u ** i) * (d ** (j - i))
            
            # Ensure we use scalar values in maximum comparison
            option_values[i, j] = max(option_value_if_held.item(), option_value_if_exercised)
    
    return option_values[0, 0]

def straddle_profit(S, K, T, r, sigma, steps=100):
    call_price = binomial_tree_american(S, K, T, r, sigma, 'call', steps)
    put_price = binomial_tree_american(S, K, T, r, sigma, 'put', steps)
    total_cost = call_price + put_price
    
    profit_up = max(S + (S * 0.2) - K, 0) + max(K - (S + (S * 0.2)), 0) - total_cost
    profit_down = max(K - (S - (S * 0.2)), 0) + max((S - (S * 0.2)) - K, 0) - total_cost
    profit_no_move = -total_cost
    
    return profit_up, profit_down, profit_no_move, total_cost

def objective(K, S, T, r, sigma, steps=100):
    profit_up, profit_down, profit_no_move, _ = straddle_profit(S, K, T, r, sigma, steps)
    return - (profit_up + profit_down + profit_no_move)  # Minimize negative profit (maximize profit)

def sharpe_ratio(profits, risk_free_rate):
    expected_return = np.mean(profits)
    std_dev = np.std(profits)
    return (expected_return - risk_free_rate) / std_dev

# Define list of 50 equities
equities = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'BRK-B', 'V', 'JNJ', 'WMT', 
    'JPM', 'PG', 'UNH', 'DIS', 'NVDA', 'HD', 'MA', 'VZ', 'PYPL', 'ADBE', 
    'NFLX', 'INTC', 'KO', 'PFE', 'CSCO', 'PEP', 'T', 'MRK', 'ABT', 'XOM', 
    'NKE', 'MCD', 'CRM', 'LLY', 'MDT', 'AMGN', 'NEE', 'BA', 'COST', 'AVGO', 
    'IBM', 'HON', 'ACN', 'TMO', 'MMM', 'TXN', 'UNP', 'QCOM', 'LOW'
]
equities = ['AAPL', 'MSFT', 'GOOGL']

lag_period = 30

# Initialize StraddleSelector and perform initial calculations
straddle_selector = StraddleSelector(equities, lag_period)
straddle_selector.initial_calculations()

# Run for a fixed number of iterations
straddle_selector.run_iterations(num_iterations=1)


INFO:root:Initial analysis for pair: AAPL, MSFT
/Users/kavitakar/MS&E 244 Final Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/var/folders/jn/h8_6ldvx1wz2fwvbpv9k_nmh0000gn/T/ipykernel_75797/69547978.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.p_values_df = pd.concat([self.p_values_df, new_row], ignore_index=True)
INFO:root:Initial analysis for pair: AAPL, GOOGL
/Users/kavitakar/MS&E 244 Final Project/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but 

In [ ]:
#!pip install scipy numpy pandas yfinance statsmodels scikit-learn websocket-client
